# YOLO v8 - hyperparameter tuning

Sources:
- https://github.com/ultralytics/ultralytics
- https://docs.ultralytics.com/guides/hyperparameter-tuning/#best_hyperparametersyaml

Hyperparameter tuning is used for optimizing the model's performance. It is aimed at optimizing metrics such as accuracy, precision, and recall. In the context of Ultralytics YOLO, these hyperparameters could range from learning rate to architectural details (number of layers or types of activation functions used).

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json

import os
import glob as glob
import matplotlib.pyplot as plt
import random
import cv2
import torch


In [2]:
# Check the files in current directory & delete them if neccessary
!ls

# # remove directories and files
# !rm -rf ./garbage
# !rm -rf ./garbage_sub
# !rm -rf ./runs

# !rm -rf ./yolov8n.pt

# !rm -rf ./fetch_data.sh
# !rm -rf ./requirements.txt

sample_data


In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
device

device(type='cuda')

In [5]:
# Install the ultralytics package
!pip install ultralytics -qq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 660.5/660.5 kB 7.5 MB/s eta 0:00:00


In [6]:
from ultralytics import YOLO

### Download the data
- have to specify the format (yolo)

In [7]:
# DATA
!wget -O fetch_data.sh https://raw.githubusercontent.com/aml-2023/final-project/main/fetch_data.sh
!bash fetch_data.sh --type yolo --output garbage

--2023-12-13 20:21:05--  https://raw.githubusercontent.com/aml-2023/final-project/main/fetch_data.sh
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2386 (2.3K) [text/plain]
Saving to: ‘fetch_data.sh’

fetch_data.sh       100%[===================>]   2.33K  --.-KB/s    in 0s      

2023-12-13 20:21:06 (29.9 MB/s) - ‘fetch_data.sh’ saved [2386/2386]

yolo dataset
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   894  100   894    0     0   2807      0 --:--:-- --:--:-- --:--:--  2802
100  274M  100  274M    0     0  96.6M      0  0:00:02  0:00:02 --:--:--  130M
Data downloaded and extracted into garbage


### Subset the data

In [9]:
# Function that takes a subset of the data

import os
import shutil
from glob import glob
import numpy as np
import pathlib

def label_path_from_image_path(image_path: str, base_path):
    """Gets the YOLO label path from the image path."""
    label = image_path.split("/")[-1]
    label = label[:-3] + "txt"
    label = os.path.join(base_path, label)
    return label


def get_image_label_path_pair(base_path: str, split_folder: str):
    """Gets all the image and label path pairs for a specific base path and the split folder, e.g. garbage and test."""
    img_path = os.path.join(base_path, split_folder, "images")
    labels_path = os.path.join(base_path, split_folder, "labels")

    pairs = []
    for image_name in glob(f"{img_path}/*.jpg"):
        label = label_path_from_image_path(image_name, labels_path)
        pairs.append((image_name, label))

    return pairs


def subset_split_folder(yolo_root_dir: str, percentage: float, split_folder: str, out_dir: str):
    """Subsets a split folder (train, test, valid) and copies the subset to a new directory."""
    pairs = get_image_label_path_pair(yolo_root_dir, split_folder)
    subset_len = int(len(pairs) * percentage)
    subset_idx = np.random.randint(low=0, high=len(pairs), size=subset_len)

    subset_pairs = [pairs[i] for i in subset_idx]

    out_dir_img = os.path.join(out_dir, split_folder, "images")
    out_dir_labels = os.path.join(out_dir, split_folder, "labels")

    pathlib.Path(out_dir_img).mkdir(parents=True, exist_ok=True)
    pathlib.Path(out_dir_labels).mkdir(parents=True, exist_ok=True)

    for img, label in subset_pairs:
        dest_img_path = os.path.join(out_dir_img, img.split("/")[-1])
        dest_label_path = os.path.join(out_dir_labels, label.split("/")[-1])

        shutil.copy(img, dest_img_path)
        shutil.copy(label, dest_label_path)

def subset_yolo_data(yolo_root_dir: str, percentage: float, out_dir: str):
    """Subsets the YOLO dataset by taking a percentage of the original data and moving it into a new directory.

    :arg
        yolo_root_dir (str): the root directory where the yolo data is.
        percentage (float): the percentage of images to keep.
        out_dir (str): the output directory, will be created if it does not exist.
    """
    subset_split_folder(yolo_root_dir, percentage, "train", out_dir)
    subset_split_folder(yolo_root_dir, percentage, "test", out_dir)
    subset_split_folder(yolo_root_dir, percentage, "valid", out_dir)

    other_files = ["README.dataset.txt", "README.roboflow.txt", "data.yaml"]

    for file in other_files:
        old_path = os.path.join(yolo_root_dir, file)
        new_path = os.path.join(out_dir, file)
        shutil.copy(old_path, new_path)

In [10]:
# Subset the data
subset_yolo_data("garbage", 0.10, "garbage_sub")

We use the model.tune() method for hyperparameter tuning. We have set the epochs to 25, the iterations to 10, and the optimizer to AdamW. We skip plotting, checkpointing, and validation other than on the final epoch for faster Tuning.

In [11]:
# Initialize the YOLO model
model = YOLO('yolov8n.pt')

# Tune hyperparameters on the subset
model.tune(data="/content/garbage_sub/data.yaml", epochs=25, iterations=10, optimizer='AdamW', plots=False, save=False, val=False)

100%|██████████| 6.23M/6.23M [00:00<00:00, 116MB/s]

Tuner: Initialized Tuner instance with 'tune_dir=runs/detect/tune'
Tuner: 💡 Learn about tuning at https://docs.ultralytics.com/guides/hyperparameter-tuning
Tuner: Starting iteration 1/10 with hyperparameters: {'lr0': 0.01, 'lrf': 0.01, 'momentum': 0.937, 'weight_decay': 0.0005, 'warmup_epochs': 3.0, 'warmup_momentum': 0.8, 'box': 7.5, 'cls': 0.5, 'dfl': 1.5, 'hsv_h': 0.015, 'hsv_s': 0.7, 'hsv_v': 0.4, 'degrees': 0.0, 'translate': 0.1, 'scale': 0.5, 'shear': 0.0, 'perspective': 0.0, 'flipud': 0.0, 'fliplr': 0.5, 'mosaic': 1.0, 'mixup': 0.0, 'copy_paste': 0.0}


Saved runs/detect/tune/tune_scatter_plots.png
Saved runs/detect/tune/tune_fitness.png

Tuner: 1/10 iterations complete ✅ (186.46s)
Tuner: Results saved to runs/detect/tune
Tuner: Best fitness=0.24564 observed at iteration 1
Tuner: Best fitness metrics are {'metrics/precision(B)': 0.57573, 'metrics/recall(B)': 0.60714, 'metrics/mAP50(B)': 0.51891, 'metrics/mAP50-95(B)': 0.21528, 'val/box_loss': 2.30955, 'val/cls_loss': 1.80074, 'val/dfl_loss': 3.01023, 'fitness': 0.24564}
Tuner: Best fitness model is runs/detect/train
Tuner: Best fitness hyperparameters are printed below.

Printing 'runs/detect/tune/best_hyperparameters.yaml'

lr0: 0.01
lrf: 0.01
momentum: 0.937
weight_decay: 0.0005
warmup_epochs: 3.0
warmup_momentum: 0.8
box: 7.5
cls: 0.5
dfl: 1.5
hsv_h: 0.015
hsv_s: 0.7
hsv_v: 0.4
degrees: 0.0
translate: 0.1
scale: 0.5
shear: 0.0
perspective: 0.0
flipud: 0.0
fliplr: 0.5
mosaic: 1.0
mixup: 0.0
copy_paste: 0.0

Tuner: Starting iteration 2/10 with hyperparameters: {'lr0': 0.00931, 'lrf':

From the results above, we can see that the best fitness (0.30269) was observed during the 5th iteration.

We can also see the optimal parameters.

**Best fitness metrics:**

{'metrics/precision(B)': 0.78171, 'metrics/recall(B)': 0.5, 'metrics/mAP50(B)': 0.59134, 'metrics/mAP50-95(B)': 0.27062, 'val/box_loss': 1.72724, 'val/cls_loss': 1.49193, 'val/dfl_loss': 2.09641, 'fitness': 0.30269}



After we complete the hyperparameter tuning, we get several files:
- **best_hyperparameters.yaml** --> A YAML file that contains the optimal hyperparameters that we can later use to initialize future trainings.
- **best_fitness.png** --> A plot displaying fitness (typically a performance metric like AP50) against the number of iterations.
- **tune_results.csv** --> A CSV file containing the results (metrics) of each iteration.
- **tune_scatter_plots.png** --> A file that contains scatter plots generated from tune_results.csv, which can help visualize the relationships between the different hyperparameters and performance metrics.
- **weights/** --> A directory thatcontains the saved PyTorch models for the last and the best iterations during the hyperparameter tuning proces**
